<h1><center>Лабораторна робота 6.</center></h1>
<h2><center>Лінійна регресія, Lasso і RF-регресія для задачі визначення якості вина</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

<a class="anchor" id="lab-6"></a>

## Зміст

- [6.1. Завантаження та обробка даних](#lab-6.1)
- [6.2. Лінійний регресійний аналіз](#lab-6.2)  
- [6.3. Регресійний аналіз із регуляризацією](#lab-6.3)
- [6.4. Побудова ансамблевих моделей](#lab-6.4)
- [6.5. Оцінювання та порівняння моделей](#lab-6.5)
- [6.6. Покращення та подальший тюнинг моделей](#lab-6.6)

In [6]:
# відключимо попередження Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split

<a class="anchor" id="lab-6.1"></a>

## <span style="color:blue; font-size:1.2em;">6.1. Завантаження та обробка даних</span>

[Повернутися до змісту](#lab-6)

В цій лабораторній роботі будемо працювати з набором даних за якістю білого вина ([репозиторій UCI](https://archive.ics.uci.edu/ml/datasets/wine+quality)).

Набір даних "Wine Quality UCI" включає дані, що стосуються червоних і білих сортів португальського вина "Vinho Verde". Цей набір даних є унікальним, оскільки він фокусується виключно на фізико-хімічних та сенсорних змінних, не включаючи інформацію про типи винограду, марки вина та ціни продажу. Набори даних слугують для задач класифікації або регресії, із класами, які є впорядкованими, але не збалансованими, що вказує на переважання нормальних вин над відмінними або поганими. Релевантність усіх вхідних змінних не є однозначною, що робить застосування методів відбору ознак до цієї задачі досить актуальним.

Набір даних складається з 11 неперервних вхідних змінних і однієї вихідної змінної:

1. Fixed Acidity (Фіксована кислотність) - відображає кількість нелетких кислот у вині.
2. Volatile Acidity (Летюча кислотність) - кількість оцтової кислоти у вині, яка при занадто високому рівні може призвести до неприємного, схожого на оцет, смаку.
3. Citric Acid (лимонна кислота) - у невеликих кількостях лимонна кислота може додавати вину "свіжості" та аромату.
4. Residual Sugar (залишковий цукор) - кількість цукру, що залишається після припинення ферментації, рідко можна зустріти вина з вмістом цукру менше 1 грама/літр, а вина з вмістом цукру понад 45 грамів/літр вважаються солодкими.
5. Chlorides (хлориди) - кількість солі у вині.
6. Free Sulfur Dioxide (вільний діоксид сірки) - вільна форма SO2 існує в рівновазі між молекулярним SO2 (у вигляді розчиненого газу) і бісульфіт-іоном; він запобігає розмноженню мікроорганізмів і окисленню вина.
7. Total Sulfur Dioxide (загальний діоксид сірки) - сума вільної та зв'язаної форм S02; у низьких концентраціях SO2 майже не відчувається у вині, але при концентрації вільного SO2 понад 50 проміле, SO2 стає помітним у носі та смаку вина.
8. Density (щільність) - густина вина близька до густини води і залежить від вмісту алкоголю і цукру.
9. pH - описує кислотність або лужність вина за шкалою від 0 (дуже кисле) до 14 (дуже лужне); більшість вин знаходиться між 3-4 балами за шкалою pH.
10. Sulphates (сульфати) - добавка до вина, яка може сприяти підвищенню рівня діоксиду сірки (S02), що діє як антимікробний засіб і антиоксидант.
11. Alcohol (алкоголь) - відсотковий вміст алкоголю у вині.
12. Quality (якість) - вихідна змінна (цільова змінна), що базується на сенсорних даних і оцінюється від 0 до 10 балів.


Детальний опис цього набору даних наведено у науковій роботі [Using Data Mining for Wine Quality Assessment](https://www.researchgate.net/publication/221612614_Using_Data_Mining_for_Wine_Quality_Assessment)

In [7]:
winequality_white_url = 'https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-06_regression-wine/winequality-white.csv'
data = pd.read_csv(winequality_white_url, sep=';')

In [8]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Виконаємо відокремлення цільової ознаки. Для цього розділимо навчальну вибірку за відношенням 7:3; тут 30% даних - валідаційна вибірка. Задамо `random_state=17` і виконаємо стандартизацію даних за допомогою `StandardScaler()`. Надалі працюйте зі змінними `X_train_scaled` та `X_test_scaled`.

In [10]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

<a class="anchor" id="lab-6.2"></a>

## <span style="color:blue; font-size:1.2em;">6.2. Лінійний регресійний аналіз</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 1</span>

**Для всіх варіантів:**

**Побудуйте модель простої лінійної регресії за вхідними даними оцінки якості білого вина. Майте на увазі, що для навчання та тестування моделі треба використати стандартизовані значення незалежних змінних `X_train_scaled` та `X_test_scaled`, і початкові значення цільової ознаки `y_train` та `y_test`.**

**Для варіантів 1–2:**

Додайте до побудованої моделі поліноміальні ознаки за допомогою `sklearn.preprocessing.PolynomialFeatures` для ускладнення моделі. Протестуйте різні ступені (наприклад, 2, 3) й оцініть продуктивність моделі за допомогою `sklearn.linear_model.Ridge` для контролю за перенавчанням. Інтерпретуйте, як степені полінома впливають на точність моделі та її перенавчання, а також на ефект регуляризації від штрафів.

**Для варіантів 3–4:**

Виконайте оцінювання впливу різних методів масштабування (StandardScaler та MinMaxScaler) на продуктивність лінійної регресійної моделі. Використайте `sklearn.preprocessing.StandardScaler` та `MinMaxScaler` для попередньої обробки даних, а потім застосуйте лінійну регресійну модель. Оцініть модель за допомогою метрик RMSE та коефіцієнт детермінаї (R^2), щоб зрозуміти вплив нормалізації на продуктивність моделі.

**Для варіантів 5–6:**

Реалізуйте покрокову регресію, використовуючи комбінацію прямого та зворотного відбору ознак. Використайте `sklearn.feature_selection.RFE` для зворотного виключення і вручну додайте ознаки для прямого відбору. Оцініть роботу моделі на кожному кроці, щоб визначити найбільш значущі ознаки.

**Для варіантів 7–8:**

Застосуйте регуляризацію ElasticNet (`sklearn.linear_model.ElasticNet`), щоб зрозуміти компроміс між штрафами за Ridge та Lasso. Змінюйте співвідношення l1_ratio для перемикання між ефектами Ridge та Lasso й визначте, як це впливає на продуктивність моделі та коефіцієнти ознак.

**Для варіантів 9–10:**

Визначте викривлені ознаки та застосуйте лог-перетворення за допомогою numpy.log1p. Побудуйте лінійну регресійну модель і порівняйте її продуктивність із початковою моделлю, щоби побачити ефект від перетворення викривлених даних.

**Для варіантів 11–12:**

Використайте PolynomialFeatures з параметром interaction_only=True для створення умов взаємодії між функціями. Проаналізуйте з допомогою програмного коду, як ці взаємодії впливають на продуктивність моделі і які з них є найбільш значущими.

**Для варіантів 13–14:**

Застосуйте `sklearn.preprocessing.QuantileTransformer` для перетворення ознак у більш подібний до гаусівського розподіл. Оцініть за допомогою програмного коду, як це перетворення впливає на продуктивність регресії.

**Для варіантів 15–16:**

Застосуйте `sklearn.linear_model.RANSACRegressor` для надійної регресії (robust regression), щоб зменшити вплив пропусків. Порівняйте його продуктивність зі стандартною лінійною регресією, щоб зрозуміти вплив надійних методів на регресію.

**Для варіантів 17–18:**

Використайте LassoCV для відбору ознак, який поєднує регресію Lasso з cross-validation. У такий спосіб визначте найбільш значущі ознаки, контролюючи при цьому надмірне перенавчання моделі.

**Для варіантів 19–20:**

Після навчання моделі перегляньте її коефіцієнти й відсортуйте ознаки за впливом на якість вина. Майте на увазі, що великі за модулем негативні значення коефіцієнтів теж вказують на сильний вплив на цільову ознаку. У такий спосіб виясніть, яка ознака в моделі лінійної регресії має найбільший вплив на якість вина.

<a class="anchor" id="lab-6.3"></a>

## <span style="color:blue; font-size:1.2em;">6.3. Регресійний аналіз із регуляризацією</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 2</span>

**Для всіх варіантів:**

**Побудуйте моделі Lasso-регресі з параметрами `alpha=0.01`, `random_state=17` та `cv=5`. Обрахуйте вплив усіх незалежних ознак на цільову ознаку Lasso-регресії. Відсортуйте ознаки за впливом на якість вина. Майте на увазі, що для навчання та тестування моделі треба використати стандартизовані значення незалежних змінних `X_train_scaled` та `X_test_scaled`, і початкові значення цільової ознаки `y_train` та `y_test`.**

**Для варіантів 1–2:**

Використайте LassoCV для автоматичного вибору найкращого значення параметра `alpha` через `cross_validation`. Проаналізуйте, як різні значення параметра `alpha` впливають на розрідженість розв’язку та загальну точність моделі.

**Для варіантів 3–4:**

Обрахуйте значення середньоквадратичної помилки та коефіцієнта детермінації для побудованої Lasso-регресії за навчальною та тестовою вибірками. Оцініть продуктивність моделі за цими показниками.

**Для варіантів 5–6:**

Поєднайте Lasso-регресію з поліноміальним розширенням ознак. Визначте баланс між складністю ознак та ефектом регуляризації Lasso.

**Для варіантів 7–8:**

Застосуйте регресію Lasso та Ridge (RidgeCV) за тестовим набором даних. Використовуючи це порівняння, визначте відмінності в тому, як Lasso і Ridge обробляють мультиколінеарність і важливість ознак.

**Для варіантів 9–10:**

Застосуйте три різних методи нормалізації до вхідних даних перед регресією Lasso і проаналізуйте, як вони впливають на продуктивність моделі.

**Для варіантів 11–12:**

Створіть кілька bootstrapped-вибірок початкового набору даних і застосуйте Lasso-регресію до кожної з них. Використовуючи цей підхід, оцініть стабільність виділення ознак за допомогою Lasso за різними bootstrapped-вибірками.

**Для варіантів 13–14:**

Створіть підходи взаємодії (див. лекційну записку 6) та застосуйте регресію Lasso до них. Дослідіть, які взаємодії є найбільш впливовими і як вони впливають на передбачуваність моделі.

**Для варіантів 15–16:**

Проведіть експерименти з побудованою Lasso-регресію з різними функціями втрат, використовуючи `sklearn.linear_model.SGDRegressor` зі штрафом l1. Оцініть, як різні функції втрат впливають на регуляризаційний ефект Lasso.

**Для варіантів 17–18:**

Реалізуйте GroupLasso-регресію за допомогою GroupLassoCV. З цим підходом визначте рівень мультиколінеарності між ознаками та виберіть найбільш впливові ознаки.

**Для варіантів 19–20:**

Обрахуйте найкраще значення `alpha` для Lasso-регресії за 5-кратною крос-валідацією. Використайте LassoCV, `random_state = 17` nf команду `alpha_`. Визначте, яка ознака в моделі Lasso-регресії має найбільший вплив на якість вина.


<a class="anchor" id="lab-6.4"></a>

## <span style="color:blue; font-size:1.2em;">6.4. Побудова ансамблевих моделей</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 3</span>

**Для варіантів 1–2:**

Використайте GridSearchCV для оцінювання значень гіперпараметрів моделі Random Forest, таких як `n_estimators`, `max_depth` та `min_samples_split`, за поданим набором даних визначення якості вина.

**Для варіантів 3–4:**

Застосуйте Random Forest та Extra Trees (`sklearn.ensemble.ExtraTreesRegressor`) за поданим набором даних. Порівняйте їх результати, щоб зрозуміти відмінності та подібності між цими ансамблевими методами.

**Для варіантів 5–6:**

Порівняйте показники ROC AUC для моделей беггінгу логістичної регресії, беггінгу дерев рішень та звичайного випадкового лісу.

**Для варіантів 7–8:**

Використайте `BalancedRandomForestClassifier` з imbalanced-learn для обробки незбалансованих класів у вхідному наборі даних.

**Для варіантів 9–10:**

Визначте вплив різної глибини дерев у випадковому лісі. Виконайте експерименти та дослідіть, як дрібні та глибокі дерева впливають на точність та узагальнення моделі.

**Для варіантів 11–12:**

Проведіть експерименти з різними критеріями розбиття (`gini` vs. `entropy`) у випадковому лісі. Це покаже, як вибір критерію впливає на продуктивність моделі та структуру дерева.

**Для варіантів 13–14:**

Використайте методи відбору ознак, такі як `SelectFromModel` з Lasso або RFE (Recursive Feature Elimination) перед застосуванням Random Forest. Оцініть як попередньо відібрані ознаки впливають на точність та обчислювальну ефективність моделі.

**Для варіантів 15–16:**

Проведіть експерименти з різними стратегіями вибірки для створення дерев, як от bootstrapping та random subsampling without replacement. Проаналізуйте, як ці стратегії впливають на зміщення та дисперсію моделі.

**Для варіантів 17–18:**

Побудуйте модель беггінгу логістичної регресії та беггінгу дерев рішень для поданої задачі визначення якості вина. Порівняйте середньоквадратичне відхилення оцінки точності в моделях беггінгу логістичної регресії, дерев рішень та звичайного випадкового лісу.

**Для варіантів 19–20:**

Використайте модель випадкового лісу з балансуванням ваги класу (class weight balancing). Порівняйте продуктивність моделі випадкового лісу зі bootstrap samples та без них.


<a class="anchor" id="lab-6.5"></a>

## <span style="color:blue; font-size:1.2em;">6.5. Оцінювання та порівняння моделей</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 4</span>


**Для варіантів 1–2:**

Використайте перехресну оцінку (`cross_val_score`) та криву навчання (`sklearn.model_selection.learning_curve`) для порівняння побудованих моделей.

**Для варіантів 3–4:**

Реалізуйте порівняння побудованих моделей за допомогою інформаційних критеріїв, таких як AIC (Akaike Information Criterion) і BIC (Bayesian Information Criterion).

**Для варіантів 5–6:**

Порівняйте ефективність звичайної моделі логістичної регресії з ансамблевими методами, зокрема, Bagging, Boosting та Stacking. Використайте класи sklearn.ensemble для реалізації цих підходів та оцініть їхню продуктивність проти логістичної регресії.

**Для варіантів 7–8:**

Проведіть тестування ефективності рекурсивного виключення ознак у поєднанні з перехресною перевіркою для відбору найбільш впливових ознак у моделі беггінгу логістичних регресій. Для цього реалізуйте RFECV з `sklearn.feature_selection`. Проаналізуйте атрибут cv_results_, щоб зрозуміти варіацію продуктивності з різною кількістю ознак. Оцініть важливість остаточно відібраних ознак та їхній внесок у продуктивність моделі.

**Для варіантів 9–10:**

Порівняйте моделі на основі важливості, яку вони надають різним ознакам. Використайте метод важливості перестановок (`sklearn.inspection.permutation_importance`), щоб оцінити вплив кожної ознаки в різних моделях.

**Для варіантів 11–12:**

Оцініть стабільність моделі за допомогою bootstrapping. Створіть кілька вибірок bootstrapping та оцініть продуктивність кожної моделі на цих вибірках. Оцініть стійкість і чутливість моделі до варіацій даних.

**Для варіантів 13–14:**

Обрахуйте важливість кожної ознаки за допомогою важливості перестановок (permutation importance). Для цього використайте модель випадкового лісу та застосуйте `permutation_importance` з `sklearn.inspection`. Випадковим чином перетасуйте кожну ознаку в наборі даних і обчисліть зміну продуктивності моделі. Тут суттєве падіння продуктивності моделі буде вказувати на високу важливість ознаки. Повторіть цей процес кілька разів та обрахуйте середнє значення поміж отриманих значень.

**Для варіантів 15–16:**

Використайте ROC-AUC та криві Precision-Recall для оцінювання та порівняння продуктивності моделей, що особливо корисно для незбалансованих наборів даних. Обрахуйте середньоквадратичні помилки налаштованого випадкового лісу за навчальною та валідаційною вибірками та на крос-валідації (cross_val_score з `scoring='neg_mean_squared_error'`).

**Для варіантів 17–18:**

Проведіть аналіз чутливості моделей з попереднього до їх гіперпараметрів. Використайте GridSearchCV для аналізу того, як різні налаштування гіперпараметрів впливають на продуктивність моделі.

**Для варіантів 19–20:**

Оцініть важливість ознак в моделі випадкового лісу. Використайте атрибут feature_importances_ моделі для отримання рейтингу ознак на основі їхньої важливості.
Порівняйте ці значення важливості з результатами моделі беггінгу логістичних регресій.
Візуалізуйте важливість ознак, що отримані з моделі випадкового лісу, за допомогою гістограм бібліотеки `MatPlotLib`.


<a class="anchor" id="lab-6.6"></a>

## <span style="color:blue; font-size:1.2em;">6.6. Покращення та подальший тюнинг моделей</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 5</span>

**Для варіантів 1–2:**

Використайте інші ансамблеві методи, як от, Gradient Boosting (`sklearn.ensemble.GradientBoostingRegressor`) та AdaBoost (`sklearn.ensemble.AdaBoostRegressor`). Проаналізуйте, як ці методи покращують продуктивність моделі порівняно з беггінгами.

**Для варіантів 3–4:**

Реалізуйте нелінійні моделі, такі як Kernel SVM (`sklearn.svm.SVC` з нелінійними ядрами) та K-Nearest Neighbors (`sklearn.neighbors.KNeighborsRegressor`). Порівняйте їхню продуктивність зі звичайною логістичною регресією, щоб зрозуміти переваги врахування нелінійних зв’язків.

**Для варіантів 5–6:**

Реалізуйте ансамбль за допомогою різних ансамблевих методів із попереднього завдання та оцініть його точність. Використайте метод голосування або усереднення, щоб об’єднати результати із цих ансамблевих моделей. Експериментуйте з різними схемами зважування для прогнозів кожної моделі. Оцініть ефективність за допомогою перехресної перевірки.

**Для варіантів 7–8:**

Протестуйте стійкість моделі, ввівши шум у початковий набір даних. Оцініть, як моделі беггінгу логістичних регресій та випадкового лісу працюють при різних рівнях шуму, підкреслюючи їхню стійкість до проблем із якістю даних.

**Для варіантів 9–10:**

Експериментуйте з різними функціями втрат, особливо для регресійних моделей. Порівняйте моделі, що використовують середньоквадратичну похибку, середню абсолютну похибку та втрати Губера, щоб зрозуміти, як вибір функції втрат впливає на прогнози моделей Lasso-регресії та випадкового лісу.

**Для варіантів 11–12:**

Впроваджувати методи функціональної інженерії, такі як цільове кодування (target encoding) або категоріальне вбудовування (categorical embedding). Проаналізуйте, як ці методи підвищують точність моделей із попередніх завдань порівняно з базовими методами кодування.

**Для варіантів 13–14:**

Реалізуйте сучасні алгоритми градієнтного бустингу, як от, XGBoost, LightGBM та CatBoost. Налаштуйте гіперпараметри для кожної моделі. Оцініть продуктивність кожної моделі, використовуючи метрики, що відповідають задачі (accuracy, recall, precision, F1-score). Порівняйте моделі на основі продуктивності, часу навчання та складності моделі. Проаналізуйте важливість ознак, що надаються цими моделями.

**Для варіантів 15–16:**

Використайте методи байєсівської оптимізації для пошуку оптимальних гіперпараметрів для моделей із попередніх завдань. Порівняйте ефективність цього підходу з традиційними методами GridSearchCV та RandomizedSearchCV.

**Для варіантів 17–18:**

Застосуйте методи пониження розмірності, такі як t-SNE (`sklearn.manifold.TSNE`) та PCA (`sklearn.manifold.PCA`) для візуалізації новостворених ознак. Оновіть моделі з попередніх завдань за новими даними з пониженою розмірністю. Оцініть вплив цих методів пониження розмірності на продуктивність та інтерпретованість моделей із завдання 4.

**Для варіантів 19–20:**

Застосуйте різні методи для обробки незбалансованих наборів даних, такі як SMOTE (`imblearn.over_sampling.SMOTE`) або коригування ваги класів. Оцініть, як ці методи впливають на продуктивність моделі для незбалансованих сценаріїв.